In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingRegressor

In [12]:
cas_1 = pd.read_csv("./cas1.csv")
cas_2 = pd.read_csv("./cas2.csv")
cas_3 = pd.read_csv("./cas3.csv")

In [97]:
final_22 = pd.concat([cas_1, cas_2, cas_3])

In [98]:
final_22 = final_22.set_index("INDEX")

In [99]:
df_14_22 = pd.read_csv("../DATASETS_FINALS/TOT_14_22.csv")
df_14_22 = df_14_22.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [100]:
df_14_22 = df_14_22.set_index("INDEX")

In [101]:
df_14_22.loc[final_22.index, "PRODUCCION"] = final_22["PRODUCCION"]

In [102]:
df_14_22 = df_14_22.drop(columns ="SUPERFICIE")

In [173]:
df_14_22_tr = df_14_22[(df_14_22["ID_FINCA"] != "13450") |
                       ((df_14_22["CAMPAÑA"] == "14") & (df_14_22["ID_FINCA"] == "13450"))]
df_13450 = df_14_22[(df_14_22["ID_FINCA"] == "13450") & (df_14_22["CAMPAÑA"] == "22")]

X = df_14_22_tr.drop(columns = "PRODUCCION")
#X = df_14_22_tr["ID_FINCA"]
y = df_14_22_tr["PRODUCCION"]

X_pred = df_13450.drop(columns = "PRODUCCION")
#X_pred = df_13450["ID_FINCA"]

In [174]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["CAMPAÑA", "ID_FINCA", "ID_ZONA", "ALTITUD", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])
X = encoder.fit_transform(X)
X_pred_enc = encoder.transform(X_pred)


In [73]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator= GradientBoostingRegressor(),
                param_grid={"n_estimators": [50, 100, 120],
                            "learning_rate": [0.15],
                            "max_depth": [20, 40, 60],
                            "min_samples_leaf": [1, 2],
                            "min_impurity_decrease": [0, 0.05],
                            "min_samples_split": [5]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "r2", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    return gsc

In [ ]:
grid_ = grid(X, y)

In [175]:
import xgboost as xgb
model_grid = GradientBoostingRegressor(**grid_.best_params_)
model_grid.fit(X,y)
X_pred["PRODUCCION"] = model_grid.predict(X_pred_enc)

In [176]:
df_14_22_final = pd.concat([df_14_22_tr, X_pred])

In [177]:
df_14_22_final

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,CAMPAÑA_ESTACION
INDEX,,,,,,,,,,,,,
0,14,76953,515,4,660,26,2,0,1,22215.000000,660,0,2014_4
1,14,84318,515,4,660,26,2,0,1,22215.000000,660,0,2014_4
2,14,85579,340,4,520,32,2,0,1,20978.000000,520,0,2014_4
3,14,69671,340,4,520,32,2,0,1,40722.000000,520,0,2014_4
4,14,14001,852,14,650-660,81,1,0,1,14126.000000,650,10,2014_14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9597,22,58769,239,6,700,32,2,0,1,39431.617000,700,0,2022_6
9598,22,58769,239,6,700,59,2,0,1,35431.156000,700,0,2022_6
9599,22,88928,239,6,700,40,2,0,1,45362.500000,700,0,2022_6


In [178]:
df_22_final = df_14_22_final[df_14_22_final["CAMPAÑA"] == "22"]

In [180]:
df_22_final = df_22_final.sort_index()

In [183]:
df_22_final.to_csv("./final_22.csv", index=False)